In [1]:
!git clone --branch=add-data https://github.com/schmidt-jake/kaggle.git
%cd kaggle
!pip install -U pip
!pip install -U -r /kaggle/working/kaggle/requirements/main.in --extra-index-url https://download.pytorch.org/whl/cu116

Cloning into 'kaggle'...
remote: Enumerating objects: 597, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 597 (delta 151), reused 171 (delta 98), pack-reused 350
Receiving objects: 100% (597/597), 114.30 KiB | 668.00 KiB/s, done.
Resolving deltas: 100% (352/352), done.
/kaggle/working/kaggle
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.1.2
    Uninstalling pip-22.1.2:
      Successfully uninstalled pip-22.1.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 13.3 MB/s eta 0:00:00
     ━━

In [2]:
!python -m mayo_clinic_strip_ai.train roi_path=/kaggle/input/mayo-rois/ROIs.csv

[2022-08-09 13:33:31,123][__main__][INFO] - Using device: cuda:0
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100%|████████████████████████████████████████| 110M/110M [00:10<00:00, 10.6MB/s]
Adjusting learning rate of group 0 to 2.0000e-03.
[2022-08-09 13:33:51,270][__main__][INFO] - num workers: 2
[2022-08-09 13:33:51,271][__main__][INFO] - prefetch samples per worker: 256
[2022-08-09 13:33:51,271][__main__][INFO] - Starting epoch 0...
/kaggle/working/kaggle/mayo_clinic_strip_ai/data.py:183: RuntimeWarning: Mean of empty slice.
  if crop.mean() >= self.min_intersect_pct:
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Error executing job with overrides: ['roi_path=/kaggle/input/mayo-rois/ROIs.csv']
/kaggle/working/kaggle/mayo_clinic_strip_ai/data.py:183: RuntimeWarning: Mean of 